In [1]:
import posixpath as path
import shutil
import subprocess
import os
import time
from tqdm.notebook import tqdm

# Base path where model files are located
base_path = r"/Users/calva/OneDrive/Documentos/Doutorado_PUC/benchmark/OPM_Egg"
perm_path = path.join(base_path, "Permeability_Realizations")
main_data_file = "Egg_Model_ECL.DATA"
total_simulations = 101  # From PERM0 to PERM100

# Function to replace the permeability include line
def edit_data_file(original_path, new_path, perm_file_name):
    with open(original_path, "r") as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        if "PERM" in line and "INC" in line:
            new_lines.append(f"'{perm_file_name}' /\n")
        else:
            new_lines.append(line)

    with open(new_path, "w") as f:
        f.writelines(new_lines)

# Start measuring total execution time
start_time = time.time()

# Progress bar over all simulations
with tqdm(total=total_simulations, desc="Simulations", unit="sim") as pbar:
    for i in range(total_simulations):
        print(f"\n Starting simulation {i}")

        perm_file = f"PERM{i}_ECL.INC"
        sim_folder = f"simulacao_{i}"
        sim_path = path.join(base_path, sim_folder)

        os.makedirs(sim_path, exist_ok=True)

        # Copy fixed include files
        for filename in ["ACTIVE.INC", "COMPDAT.INC", "SCHEDULE_NEW.INC"]:
            src = path.join(base_path, filename)
            dst = path.join(sim_path, filename)

            if not os.path.exists(src):
                print(f"❌ File not found: {src}")
                continue

            shutil.copy(src, dst)

        # Copy specific permeability realization
        perm_file_path = path.join(perm_path, perm_file)
        if not os.path.exists(perm_file_path):
            print(f"⚠️ File {perm_file} not found. Skipping simulation {i}.")
            continue
        shutil.copy(perm_file_path, path.join(sim_path, perm_file))

        # Edit the main .DATA file to use the correct PERM file
        modified_data_file = path.join(sim_path, main_data_file)
        edit_data_file(path.join(base_path, main_data_file), modified_data_file, perm_file)

        # Convert Windows path to WSL path
        sim_path_wsl = path.join("/mnt/c", sim_path.lstrip("/"))
        wsl_command = f"cd {sim_path_wsl} && /usr/bin/flow {main_data_file}"

        # Run flow in WSL, suppressing output
        process = subprocess.Popen(
            ["wsl", "bash", "-c", wsl_command],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        process.communicate()

        if process.returncode == 0:
            print(f"✅ Simulation {i} completed successfully!")
        else:
            print(f"❌ Simulation {i} failed.")

        pbar.update(1)

# Show total execution time
total_time = time.time() - start_time
print(f"\n All simulations completed in {total_time:.2f} seconds.")


Simulations:   0%|          | 0/101 [00:00<?, ?sim/s]


 Starting simulation 0
✅ Simulation 0 completed successfully!

 Starting simulation 1
✅ Simulation 1 completed successfully!

 Starting simulation 2
✅ Simulation 2 completed successfully!

 Starting simulation 3
✅ Simulation 3 completed successfully!

 Starting simulation 4
✅ Simulation 4 completed successfully!

 Starting simulation 5
✅ Simulation 5 completed successfully!

 Starting simulation 6
✅ Simulation 6 completed successfully!

 Starting simulation 7
✅ Simulation 7 completed successfully!

 Starting simulation 8
✅ Simulation 8 completed successfully!

 Starting simulation 9
✅ Simulation 9 completed successfully!

 Starting simulation 10
✅ Simulation 10 completed successfully!

 Starting simulation 11
✅ Simulation 11 completed successfully!

 Starting simulation 12
✅ Simulation 12 completed successfully!

 Starting simulation 13
✅ Simulation 13 completed successfully!

 Starting simulation 14
✅ Simulation 14 completed successfully!

 Starting simulation 15
✅ Simulation 15 comp